# MLP

<img src='http://i.stack.imgur.com/3mnuT.png' />



## Notations

### Data

일반적인 분류문제에서 데이터는 다음 형태로 제공된다.  
$ \begin{align}
\mathscr{D} &= [ [\mathbf{x}^{(1)}, C^{(1)}], [\mathbf{x}^{(2)}, C^{(2)}], ... , [\mathbf{x}^{(N)}, C^{(N)}] ] \\
&= [\mathbf{x}^{(n)}, C^{(n)}]^{N}_{n=1}
\end{align} $

- input: $ \mathbf{x} = [x_1, x_2, ..., x_{N_i}] $  
- class: $ \mathcal{C} = \{C_1, C_2, ..., C_{N_k}\} $  
- target output: $ \mathbf{t} = [t_1, ..., t_{N_k}] $

 예)

|인구($x_1$) |면적($x_2$) |국명($C$)|
|-----------|-----------|--------|
|100000     |2000       |미국     |



예를 들어, 클래스가 {'미국', '중국', '일본'} 이라면,  
'미국'은 $ \mathbf{t} = [1, 0, 0] $  
'중국'은 $ \mathbf{t} = [0, 1, 0] $  
'일본'은 $ \mathbf{t} = [0, 0, 1] $  
으로 변환한다.

따라서 학습에 사용될 데이터 는 다음과 같이 변환한다.  
$ \mathscr{D} = [ [\mathbf{x}^{(1)}, \mathbf{t}^{(1)}], [\mathbf{x}^{(2)}, \mathbf{t}^{(2)}], ... , [\mathbf{x}^{(N)}, \mathbf{t}^{(N)}]  ] $  

### Neuron
For each neuron j,  

- output: $ o_j = \sigma(\mathtt{net}_j) $  
- activation function: $ \sigma(\cdot) $  
- weighted sum to neuron $j$: $\mathtt{net}_j = b_j + \sum_{i=1}^{N_i}{w_{ij}o_i}$  
- bias of neuron $j$: $b_j$  
- weight from neuron $i$ to neuron $j$: $w_{ij}$

### MLP

input: $ \mathbf{x} = [x_1, ..., x_i, ... , x_{N_i}] $  
hidden: $ \mathbf{h} = [h_1, ..., h_j, ... , h_{N_j}] $  
actual output: $ \mathbf{y} = [y_1, ..., y_k, ... , y_{N_k}] $  

$i$ : index for input neurons  
$j$ : index for hidden neurons  
$k$ : index for output neurons  

$h_j = \sigma(\mathtt{net}_j) = \sigma(b_j + \sum_{i=0}^{N_i}{w_{ij}x_i}) $  
$y_k = \sigma(\mathtt{net}_k) = \sigma(b_k + \sum_{j=0}^{N_j}{w_{jk}h_j}) $  

## Backpropagation

[Wikipedia](https://en.wikipedia.org/wiki/Backpropagation)  
[참고 1](http://galaxy.agh.edu.pl/~vlsi/AI/backp_t_en/backprop.html)  
[참고 2](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)  

